# Yellow Taxi Trip

In [182]:
import findspark

findspark.init()

In [183]:
import pyspark

sc = pyspark.SparkContext.getOrCreate()
spark = pyspark.SQLContext(sc)

In [184]:
LLC_DATASET = '/user/adampap/final/LLC'
VERIFONE_DATASET = '/user/adampap/final/VeriFone'
data = sc.textFile(VERIFONE_DATASET)

First task

In [185]:
from datetime import datetime

def initial_convert(row):
    splitted = row.split(',')
    date = datetime.strptime(splitted[2], '%Y-%m-%d %H:%M:%S')
    tip = splitted[13]
    return (date.strftime('%A'), date.hour), (float(tip), 1)

def calculate_avg(x):
    key, value = x
    return key, value[0]/value[1]

In [186]:
data = data.map(lambda x: initial_convert(x))\
    .reduceByKey(lambda x,y: (float(x[0])+float(y[0]), x[1]+y[1]))\
    .map(lambda x: calculate_avg(x))

data.filter(lambda x: x[0][0] == 'Friday')\
    .sortBy(lambda x: x[0][1])\
    .take(30)

[(('Friday', 0), 3.2483333333333326),
 (('Friday', 1), 3.082613636363635),
 (('Friday', 2), 3.033095238095238),
 (('Friday', 3), 2.99695652173913),
 (('Friday', 4), 4.024399999999999),
 (('Friday', 5), 2.762352941176471),
 (('Friday', 6), 2.1794047619047623),
 (('Friday', 7), 2.295478723404255),
 (('Friday', 8), 2.187718631178707),
 (('Friday', 9), 2.4455805243445687),
 (('Friday', 10), 2.540093023255814),
 (('Friday', 11), 2.463776595744681),
 (('Friday', 12), 2.664008810572687),
 (('Friday', 13), 2.4772857142857143),
 (('Friday', 14), 2.440964912280702),
 (('Friday', 15), 2.4228755364806864),
 (('Friday', 16), 2.7287242798353915),
 (('Friday', 17), 2.757255639097744),
 (('Friday', 18), 2.431467889908257),
 (('Friday', 19), 2.5324223602484484),
 (('Friday', 20), 2.6170462633451943),
 (('Friday', 21), 2.7630241935483872),
 (('Friday', 22), 2.6962660944206),
 (('Friday', 23), 2.807245762711864)]

Second task

In [187]:
def change(x):
    (day, hour), tip = x
    return hour, (day, tip)

def to_list(x):
    return [x]


def append(x, y):
    x.append(y)
    return x


def extend(x, y):
    x.extend(y)
    return x

def take_highest_avg(x):
    hour, days = x
    highest = sorted(days, key=lambda day: day[1])
    return hour, highest[-1]

In [188]:
data = data.map(lambda x: change(x))\
    .combineByKey(to_list, append, extend)\
    .map(lambda x: take_highest_avg(x))
data.sortByKey().take(30)

[(0, ('Thursday', 3.4306756756756767)),
 (1, ('Tuesday', 3.65375)),
 (2, ('Monday', 3.2804347826086957)),
 (3, ('Monday', 4.331538461538462)),
 (4, ('Thursday', 4.253636363636364)),
 (5, ('Saturday', 5.056111111111111)),
 (6, ('Saturday', 3.909767441860465)),
 (7, ('Sunday', 3.3388461538461534)),
 (8, ('Sunday', 2.5993827160493823)),
 (9, ('Monday', 2.4695454545454547)),
 (10, ('Tuesday', 2.8058874458874454)),
 (11, ('Tuesday', 2.6736186770428017)),
 (12, ('Wednesday', 2.7974911660777373)),
 (13, ('Thursday', 3.2026938775510208)),
 (14, ('Thursday', 2.745463917525774)),
 (15, ('Sunday', 2.800684931506848)),
 (16, ('Wednesday', 3.0968897637795263)),
 (17, ('Thursday', 2.8332110091743115)),
 (18, ('Saturday', 3.1996389891696753)),
 (19, ('Sunday', 2.6241304347826087)),
 (20, ('Monday', 2.663015873015873)),
 (21, ('Sunday', 3.1792982456140355)),
 (22, ('Monday', 3.2133136094674555)),
 (23, ('Sunday', 3.5758139534883724))]